In [1]:
import numpy as np
import scipy.constants as const
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [27]:
wavelength = np.linspace(1.27, 1.33, 1001) * 1e-6
theta_sweep = np.linspace(0.0, np.pi, 11)

pol_frac = np.linspace(0.0, 1.0, 11)
reshaped_pol_frac = np.ones((1, 1, len(wavelength), len(theta_sweep), len(pol_frac)))

for idd_pol_frac in range(len(pol_frac)):
              reshaped_pol_frac[:, :, :, :, idd_pol_frac] = pol_frac[idd_pol_frac]

delay = 0.5e-12 # 250 fs delay

In [5]:
def directional_coupler(splitratio  = 0.5):
  a = np.sqrt(1-splitratio)
  b = 1j*np.sqrt(splitratio)
  dc_t_matrix = np.zeros((2,2,len(wavelength), len(theta_sweep), len(pol_frac)), dtype=complex)
  dc_t_matrix[0,0,:,:,:] = a
  dc_t_matrix[0,1,:,:,:] = b
  dc_t_matrix[1,0,:,:,:] = b
  dc_t_matrix[1,1,:,:,:] = a
  return dc_t_matrix

def phase_shift(phi):
  phase_shift_t_matrix = np.zeros((2,2, len(wavelength), len(phi), len(pol_frac)), dtype=complex)
  for idd_phi, phi_val in enumerate(phi):
    phase_shift_t_matrix[0,0,:,idd_phi,:] = np.exp(-1j*phi_val)
    phase_shift_t_matrix[1,1,:,idd_phi,:] = np.exp(1j*phi_val)
  return phase_shift_t_matrix

def mzi(phi):
  return directional_coupler(0.5) @ phase_shift(phi) @ directional_coupler(0.5)

# input signal top element is TM and bottom is TE
def psr(delay, wavelength):
  psr_t_matrix = np.zeros((2,2,len(wavelength), len(theta_sweep), len(pol_frac)), dtype=complex)
  for idd_wavelength, wavelength_val in enumerate(wavelength):
    phase_1 = 2*np.pi/wavelength_val * delay * const.c
    psr_t_matrix[0,0,idd_wavelength, :, :] = np.exp(-1j*phase_1)
    psr_t_matrix[1,1,idd_wavelength, :, :] = 1
  return psr_t_matrix

In [13]:
wavelength = np.linspace(1.27, 1.33, 1001) * 1e-6
pol_frac = np.linspace(0, 1, num=11)
theta_sweep = np.linspace(0.0, np.pi, 11)
delay = 0.5e-12 # 250 fs delay

psr_tm = psr(delay, wavelength)
psr_tm.shape

phase_shifter_tm = phase_shift(theta_sweep)
phase_shifter_pi = phase_shift(np.ones(len(theta_sweep))*np.pi)
phase_shifter_pi_by_2 = phase_shift(np.ones(len(theta_sweep))*np.pi/2)
phase_shifter_0 = phase_shift(np.ones(len(theta_sweep)))

directional_coupler_50_50 = directional_coupler(0.5)

In [14]:
print(directional_coupler_50_50.shape)
print(psr_tm.shape)
print(phase_shifter_tm.shape)
print(phase_shifter_pi.shape)
print(phase_shifter_pi_by_2.shape)
print(phase_shifter_0.shape)

(2, 2, 1001, 11, 11)
(2, 2, 1001, 11, 11)
(2, 2, 1001, 11, 11)
(2, 2, 1001, 11, 11)
(2, 2, 1001, 11, 11)
(2, 2, 1001, 11, 11)


In [23]:

output_signal_psrc_port1 = directional_coupler_50_50 @ phase_shifter_tm @ directional_coupler_50_50 @ phase_shifter_pi @ directional_coupler_50_50 @ phase_shifter_pi @ directional_coupler_50_50 @ phase_shifter_0 @ psr_tm

In [24]:
output_signal_psrc_port1.shape

(2, 2, 1001, 11, 11)

In [34]:
final = output_signal_psrc_port1 @ pol_frac.reshape(len(pol_frac),1)
final.shape

(2, 2, 1001, 11, 1)

In [18]:
pol_frac.reshape(1,1,1,1,11)

array([[[[[0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]]]]])

In [ ]:
output_signal_psrc_port1 = np.zeros((len(wavelength), len(pol_frac), len(theta_sweep)))
output_signal_psrc_port0 = np.zeros((len(wavelength), len(pol_frac), len(theta_sweep)))

for idd_theta, theta_val in enumerate(theta_sweep):
  for idd_pol_frac, pol_frac_val in enumerate(pol_frac):
    input_state = np.array([[pol_frac_val],[1-pol_frac_val]])/np.sqrt(pol_frac_val**2 + (1-pol_frac_val)**2)
    for idd_wavelength, wavelength_val in enumerate(wavelength):
      response_value = directional_coupler(0.5) @ phase_shift(theta_val) @ directional_coupler(0.5) @ phase_shift(np.pi) @ directional_coupler(0.5) @ phase_shift(np.pi) @ directional_coupler(0.5) @ phase_shift(0.001) @ psr(delay, wavelength_val) @ input_state
      output_signal_psrc_port1[idd_wavelength][idd_pol_frac][idd_theta] = np.abs(response_value[0])
      output_signal_psrc_port0[idd_wavelength][idd_pol_frac][idd_theta] = np.abs(response_value[1])
max_signal_psrc_port1 = np.max(output_signal_psrc_port1, axis=0)
max_signal_psrc_port0 = np.max(output_signal_psrc_port0, axis=0)
min_signal_psrc_port1 = np.min(output_signal_psrc_port1, axis=0)
min_signal_psrc_port0 = np.min(output_signal_psrc_port0, axis=0)
plt.figure(figsize=(30,3.5))
for idd_pol_frac, pol_frac_val in enumerate(pol_frac):
  plt.subplot(1,2,1)
  plt.plot(theta_sweep/np.pi, 10*np.log10(max_signal_psrc_port1[idd_pol_frac,:]), label=f"TE fraction = {pol_frac_val*100:0.0f}%")
  plt.ylim([-20,0])
  plt.title("PSRC output port 1")
  plt.xlabel(r"$\theta$ [$\pi$]")
  plt.ylabel("Min Loss [dB]")
  plt.legend()
  plt.subplot(1,2,2)
  plt.plot(theta_sweep/np.pi, 10*np.log10(min_signal_psrc_port1[idd_pol_frac,:]), label=f"TE fraction = {pol_frac_val*100:0.0f}%")
  plt.legend()
  plt.ylim([-20,0])
  plt.title("PSRC output port 1")
  plt.xlabel(r"$\theta$ [$\pi$]")
  plt.ylabel("Max Loss [dB]")
plt.tight_layout()
plt.show()
plt.figure(figsize=(30,7))
for idd_theta, theta_val in enumerate(theta_sweep):
  plt.subplot(2,int(len(theta_sweep)/2),idd_theta+1)
  for idd_pol_frac, pol_frac_val in enumerate(pol_frac):
    plt.plot(wavelength, 10*np.log10(output_signal_psrc_port1[:,idd_pol_frac,idd_theta]), label=f"TE fraction = {pol_frac_val*100:0.2f}%")
    plt.title(f"PSRC output port 1 \n theta={theta_val/np.pi:0.1f} [pi]")
    plt.xlabel(r"Wavelength [$\mu$m]")
    plt.ylabel("Min Loss [dB]")
    plt.ylim([-20,0])
    plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.contourf(theta_sweep/np.pi, pol_frac, 10*np.log10(max_signal_psrc_port1), levels=100)
plt.colorbar()
plt.subplot(1,2,2)
plt.contourf(theta_sweep/np.pi, pol_frac, 10*np.log10(min_signal_psrc_port1), levels=100)
plt.colorbar()

In [ ]:
output_signal_psrc_port1 = np.zeros((len(wavelength), len(pol_frac), len(theta_sweep)))
output_signal_psrc_port0 = np.zeros((len(wavelength), len(pol_frac), len(theta_sweep)))

for idd_theta, theta_val in enumerate(theta_sweep):
  for idd_pol_frac, pol_frac_val in enumerate(pol_frac):
    input_state = np.array([[pol_frac_val],[1-pol_frac_val]])/np.sqrt(pol_frac_val**2 + (1-pol_frac_val)**2)
    for idd_wavelength, wavelength_val in enumerate(wavelength):
      response_value = directional_coupler(0.05) @ phase_shift(theta_val) @ directional_coupler(0.5) @ phase_shift(np.pi) @ directional_coupler(0.5) @ phase_shift(np.pi) @ directional_coupler(0.5) @ phase_shift(0.001) @ psr(delay, wavelength_val) @ input_state
      output_signal_psrc_port1[idd_wavelength][idd_pol_frac][idd_theta] = np.abs(response_value[0])
      output_signal_psrc_port0[idd_wavelength][idd_pol_frac][idd_theta] = np.abs(response_value[1])
max_signal_psrc_port1 = np.max(output_signal_psrc_port1, axis=0)
max_signal_psrc_port0 = np.max(output_signal_psrc_port0, axis=0)
min_signal_psrc_port1 = np.min(output_signal_psrc_port1, axis=0)
min_signal_psrc_port0 = np.min(output_signal_psrc_port0, axis=0)
plt.figure(figsize=(30,3.5))
for idd_pol_frac, pol_frac_val in enumerate(pol_frac):
  plt.subplot(1,2,1)
  plt.plot(theta_sweep/np.pi, 10*np.log10(max_signal_psrc_port1[idd_pol_frac,:]), label=f"TE fraction = {pol_frac_val*100:0.0f}%")
  plt.ylim([-20,0])
  plt.title("PSRC output port 1")
  plt.xlabel(r"$\theta$ [$\pi$]")
  plt.ylabel("Min Loss [dB]")
  plt.legend()
  plt.subplot(1,2,2)
  plt.plot(theta_sweep/np.pi, 10*np.log10(min_signal_psrc_port1[idd_pol_frac,:]), label=f"TE fraction = {pol_frac_val*100:0.0f}%")
  plt.legend()
  plt.ylim([-20,0])
  plt.title("PSRC output port 1")
  plt.xlabel(r"$\theta$ [$\pi$]")
  plt.ylabel("Max Loss [dB]")
plt.tight_layout()
plt.show()
plt.figure(figsize=(30,7))
for idd_theta, theta_val in enumerate(theta_sweep):
  plt.subplot(2,int(len(theta_sweep)/2),idd_theta+1)
  for idd_pol_frac, pol_frac_val in enumerate(pol_frac):
    plt.plot(wavelength, 10*np.log10(output_signal_psrc_port1[:,idd_pol_frac,idd_theta]), label=f"TE fraction = {pol_frac_val*100:0.2f}%")
    plt.title(f"PSRC output port 1 \n theta={theta_val/np.pi:0.1f} [pi]")
    plt.xlabel(r"Wavelength [$\mu$m]")
    plt.ylabel("Min Loss [dB]")
    plt.ylim([-20,0])
    plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.contourf(theta_sweep/np.pi, pol_frac, 10*np.log10(max_signal_psrc_port1), levels=100)
plt.colorbar()
plt.subplot(1,2,2)
plt.contourf(theta_sweep/np.pi, pol_frac, 10*np.log10(min_signal_psrc_port1), levels=100)
plt.colorbar()